### ForeFlight import workbook
rsavela@gmail.com
23 May, 2023

This workbook is intended to assist in the creation of a CSV
file suitable for use with ForeFlight.   It currently takes 
as input the backup file from FlightLogg.in, which has been
split into the flights and the aircraft data.

TODO:
 - Automatically split aircraft data based up ##PLANE header
 - Support for Pressurized and non-Piston Aircraft
 - Support for ForeFlight style instrument approach logging
 - Decode Category and Class more completely
 - HighPerformance and GearType not always identified correctly
 
Assumptions:
 - Night or day landings assume a takeoff, and populate fullstop landing.

In [1]:
import pandas as pd

In [4]:
flight_df = pd.read_csv('.././flight1.tsv', sep='\t')

In [5]:
aircraft_df = pd.read_csv('.././aircraft1.tsv', sep='\t', header=None)

In [6]:
aircraft_df.tail()

,0,1,2,3,4,5,6,7,8,9
59,##PLANE,N4044R,Piper,Cherokee Six,PA32,1,X,NaN,NaN,NaN
60,##PLANE,N8423PA,Piper,Saratoga,PA-32,1,X,"""High Performance""",NaN,NaN
61,##PLANE,N70483,Piper,Cub,PJ-35,3,X,NaN,NaN,NaN
62,##PLANE,N8635R,Schweizer,1-26,SGS-1-26,5,X,NaN,NaN,NaN
63,##PLANE,N1775S,Schweizer,2-32,SGS-2-32,5,X,NaN,NaN,NaN


In [7]:
flight_df.tail()

,Date,Registration,Type,Route,Total,PIC,Solo,SIC,Night,Dual Received,...,Actual Instrument,Simulated Instrument,Night Landings,Day Landings,Sim,Approaches,Person,Fuel Burn,Remarks,Flying
764,2023-04-25,N4753J,A23-19,KSPG-KCHN-KSPG,1.2,1.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
765,2023-04-27,N4753J,A23-19,KSPG-KSPG,0.4,0.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,Aaron Shoopak,NaN,NaN,NaN
766,2023-04-27,N4753J,A23-19,KSPG-KSPG,0.4,0.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Aaron Shoopak,NaN,NaN,NaN
767,2023-04-27,N4044R,PA32,KSPG-KEYW,1.8,1.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,Aaron Shoopak,NaN,NaN,NaN
768,2023-05-01,N4044R,PA32,KEYW-KSPG,1.8,1.8,NaN,NaN,NaN,NaN,...,0.3,NaN,NaN,1.0,NaN,NaN,Aaron Shoopak,NaN,NaN,NaN


In [8]:
flight_df.columns


Index(['Date', 'Registration', 'Type', 'Route', 'Total', 'PIC', 'Solo', 'SIC',
       'Night', 'Dual Received', 'Dual Given', 'Cross Country',
       'Actual Instrument', 'Simulated Instrument', 'Night Landings',
       'Day Landings', 'Sim', 'Approaches', 'Person', 'Fuel Burn', 'Remarks',
       'Flying'],
      dtype='object')

In [9]:
flight_df['Total'].sum()

879.6999999999999

### Convert to ForeFlight format

Write an aircraft table, and flights table in the format expected by ForeFlight


In [10]:
# Format the aircraft table output

ffimp_df = pd.DataFrame()
ffimp_df["AircraftID"] = aircraft_df[1]
ffimp_df["TypeCode"] = aircraft_df[4]
ffimp_df["Year"] = aircraft_df[9]
ffimp_df["Make"] = aircraft_df[2]
ffimp_df["Model"] = aircraft_df[3]
ffimp_df["Category"] = aircraft_df[5]
ffimp_df["Class"] = aircraft_df[5]
ffimp_df["GearType"] = aircraft_df[7]
ffimp_df["EngineType"] = aircraft_df[9]
ffimp_df["Complex"] = aircraft_df[7]
ffimp_df["HighPerformance"] = aircraft_df[7]
ffimp_df["Pressurized"] = aircraft_df[9]



In [11]:
# Figure out Complex object structure
#def test_map(x):
    

In [12]:
# function to map category
def map_category(x):
    if(x == 1):
        return ('Airplane')
    if(x == 3):
        return ('Airplane')
    if(x == 5): 
        return('Glider')
# And Class
def map_class(x):
    if(x == 1):
        return ("ASEL")
    if(x == 3):
        return ("ASES")
    if(x == 5): 
        return('Glider')

In [13]:
# Map Complex & HP field

def map_complex(x):
    if (type(x) is str):
        if(x == 'Complex'):
            return('true')
    if (type(x) is tuple):
        #return('X')
        if('Complex' in x):
            return('true')
    if (type(x) is float):
        return(None)
    if (type(x) is object):
        return(None)
    return

def map_hp(x):
    if (type(x) is str):
        if(x == '"High Performance"'):
            return('true')
    if (type(x) is tuple):
        #return('X')
        if('"High Performance"' in x):
            return('true')
    if (type(x) is float):
        return(None)
    if (type(x) is object):
        return('true')
    return('false')

def map_gear(x):
    if (type(x) is str):
        if(x == 'Complex'):
            return('true')
    if (type(x) is tuple):
        #return('X')
        if('Tailwheel' in x):
            return('FC')
    if (type(x) is float):
        return('fixed_tricycle')
    if (type(x) is object):
        return('fixed_tricycle')
    return('fixed_tailwheel')


In [14]:
ffimp_df['Category'] = ffimp_df['Category'].apply(map_category)
ffimp_df['Class'] = ffimp_df['Class'].apply(map_class)

In [15]:
#ffimp_df.dtypes
ffimp_df['Complex'] = ffimp_df['Complex'].apply(map_complex)
ffimp_df['HighPerformance'] = ffimp_df['HighPerformance'].apply(map_hp)
ffimp_df['GearType'] = ffimp_df['GearType'].apply(map_gear)
# Assume Piston engine and unpressurized
ffimp_df['EngineType'] = 'Piston'
ffimp_df['Pressurized'] = 'false'
ffimp_df['Year'] = 'false'

In [16]:
#ffimp_df.tail(60)


In [17]:
#ffimp_df['Complex'].unique()

In [18]:
#flight_df.columns

In [19]:
# Copy flights df
ff_lb_df = pd.DataFrame()

ff_lb_df['Date'] = flight_df['Date']
ff_lb_df['AircraftID'] = flight_df['Registration']
ff_lb_df['From'] = flight_df['Route']
ff_lb_df['To'] = flight_df['Route']
ff_lb_df['Route'] = flight_df['Route']
ff_lb_df['TimeOut'] = None #flight_df['']
ff_lb_df['TimeOff'] = None #flight_df['']
ff_lb_df['TimeOn'] = None #flight_df['']
ff_lb_df['TimeIn'] = None #flight_df['']
ff_lb_df['OnDuty'] = None #flight_df['']
ff_lb_df['OffDuty'] = None #flight_df['']
ff_lb_df['TotalTime'] = flight_df['Total']
ff_lb_df['PIC'] = flight_df['PIC']
ff_lb_df['SIC'] = flight_df['SIC']
ff_lb_df['Night'] = flight_df['Night']
ff_lb_df['Solo'] = flight_df['Solo']
ff_lb_df['CrossCountry'] = flight_df['Cross Country']
ff_lb_df['Distance'] = None #flight_df['']
ff_lb_df['DayTakeoffs'] = flight_df['Day Landings'].fillna(0.0).astype(int)
ff_lb_df['DayLandingsFullStop'] = flight_df['Day Landings'].fillna(0.0).astype(int) #flight_df['']
ff_lb_df['NightTakeoffs'] = flight_df['Night Landings'].fillna(0.0).astype(int)
ff_lb_df['NightLandingsFullStop'] = flight_df['Night Landings'].fillna(0.0).astype(int) # flight_df['']
ff_lb_df['AllLandings'] = None #flight_df[''].fillna(0.0).astype(int)
ff_lb_df['ActualInstrument'] = flight_df['Actual Instrument']
ff_lb_df['SimulatedInstrument'] = flight_df['Simulated Instrument']
ff_lb_df['HobbsStart'] = None #flight_df['']
ff_lb_df['HobbsEnd'] = None #flight_df['']
ff_lb_df['TachStart'] = None #flight_df['']
ff_lb_df['TachEnd'] =  None #flight_df['']
ff_lb_df['Holds'] = None #flight_df['']
ff_lb_df['Approach'] = flight_df['Approaches'].fillna(0.0).astype(int)
ff_lb_df['DualGiven'] = flight_df['Dual Given']
ff_lb_df['DualReceived'] = flight_df['Dual Received']
ff_lb_df['SimulatedFlight'] = flight_df['Sim']
ff_lb_df['GroundTraining'] = None #flight_df['']
ff_lb_df['InstructorName'] = None #flight_df['']
ff_lb_df['InstructorComments'] = None #flight_df['']
ff_lb_df['Person'] = flight_df['Person']
ff_lb_df['FlightReview'] = None #flight_df['']
ff_lb_df['Checkride'] = None #flight_df['']
ff_lb_df['IPC'] =  None #flight_df['']
ff_lb_df['CustomFields'] = None #flight_df['']
ff_lb_df['PilotComments'] = flight_df['Remarks'].fillna('')

In [20]:
## Create TO, From and Route from Route field
def map_from(x):
    if (type(x) is str):
        # Need to return the first part of Route
        arpt_from = x.split('-')
        return(arpt_from[0])
    return None

def map_to(x):
    if (type(x) is str):
        # Return last part of Route
        arpt_to = x.rsplit('-')
        return(arpt_to[0])
    return None

def map_route(x):
    # Strip off first and last elements from list, return contents
    if(type(x) is str):
        arpt_route = x.split('-')
        # remove [0] and [-1]
        new_route = arpt_route[1:-2]
        str_route = ' '.join(new_route)
        # then create string of remaining list, separated by spaces
        return(str_route)
    return None


In [21]:
ff_lb_df['From'] = ff_lb_df['From'].apply(map_from)
ff_lb_df['To'] = ff_lb_df['To'].apply(map_to)
ff_lb_df['Route'] = ff_lb_df['Route'].apply(map_route)

In [22]:
ff_lb_df[[ 'PIC', 'Night','CrossCountry', 'DualGiven', 'DualReceived', 'TotalTime']].sum()


PIC             823.1
Night            28.1
CrossCountry    296.5
DualGiven       242.2
DualReceived    155.8
TotalTime       879.7
dtype: float64

In [23]:
ff_lb_df.columns

Index(['Date', 'AircraftID', 'From', 'To', 'Route', 'TimeOut', 'TimeOff',
       'TimeOn', 'TimeIn', 'OnDuty', 'OffDuty', 'TotalTime', 'PIC', 'SIC',
       'Night', 'Solo', 'CrossCountry', 'Distance', 'DayTakeoffs',
       'DayLandingsFullStop', 'NightTakeoffs', 'NightLandingsFullStop',
       'AllLandings', 'ActualInstrument', 'SimulatedInstrument', 'HobbsStart',
       'HobbsEnd', 'TachStart', 'TachEnd', 'Holds', 'Approach', 'DualGiven',
       'DualReceived', 'SimulatedFlight', 'GroundTraining', 'InstructorName',
       'InstructorComments', 'Person', 'FlightReview', 'Checkride', 'IPC',
       'CustomFields', 'PilotComments'],
      dtype='object')

In [24]:
ff_lb_df[['Date', 'AircraftID', 'From', 'Route', 'To', 'DayTakeoffs', 'NightTakeoffs', 'Approach']].tail(50)
                                                                                                     
                                                                              
                                                        

,Date,AircraftID,From,Route,To,DayTakeoffs,NightTakeoffs,Approach
719,2022-10-20,N99265,KSPG,,KSPG,1,0,0
720,2022-10-28,N4753J,KSPG,,KSPG,2,0,0
721,2022-10-30,N4753J,KFXE,,KFXE,1,0,0
722,2022-11-03,N4753J,KSPG,,KSPG,0,0,0
723,2022-11-05,N4753J,KSPG,,KSPG,0,0,0
724,2022-11-06,N4753J,KSPG,,KSPG,0,0,0
725,2022-11-12,N4753J,KSPG,,KSPG,0,0,0
726,2022-11-15,N4753J,KSPG,,KSPG,2,0,0
727,2022-12-03,N4753J,KSPG,,KSPG,0,0,0
728,2022-12-04,N4753J,KSPG,,KSPG,1,0,0


In [28]:
# Foreflight expected header 
output_file = 'ForeFlight.import.csv'
with open(output_file, 'a') as f:
    f.write('ForeFlight Logbook Import,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,')
with open(output_file, 'a') as f:
    f.write(',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n')
with open(output_file, 'a') as f:
    f.write('Aircraft Table,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n')
with open(output_file, 'a') as f:
    f.write('Text,Text,Text,YYYY,Text,Text,Text,Text,Text,Text,Boolean,Boolean,Boolean,Boolean,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n')

ffimp_df.to_csv(output_file, index=False, mode='a')
    
with open(output_file, 'a') as f:
    f.write('Flights Table,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,#;type;runway;airport;comments,,,,,,,,,,,,name;role;email,,,,,,,,,,,,,,,,,,,\n')
with open(output_file, 'a') as f:
    f.write('Date,Text,Text,Text,Text,hhmm,hhmm,hhmm,hhmm,hhmm,hhmm,Decimal,Decimal,Decimal,Decimal,Decimal,Decimal,Decimal,Number,Decimal,Number,Number,Number,Number,Number,Decimal,Decimal,Decimal,Decimal,Decimal,Decimal,Number,Packed Detail,Packed Detail,Packed Detail,Packed Detail,Packed Detail,Packed Detail,Decimal,Decimal,Decimal,Decimal,Text,Text,Packed Detail,Packed Detail,Packed Detail,Packed Detail,Packed Detail,Packed Detail,Boolean,Boolean,Boolean,Boolean,Boolean,Text,Decimal,Decimal,Number,Date,DateTime,Boolean,Text\n')

ff_lb_df.to_csv(output_file, index=False, mode='a')    
    
